In [4]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pim_tmr as pim
import matrices as pim_mats
import fibernet

In [2]:
import mat73
import os
from scipy.io import loadmat

path = "C:\\Users\\saucourt\\Datasets"
file = "data_base_02_03_90deg.mat"

data = mat73.loadmat(os.path.join(path, file))
# data = loadmat(os.path.join(path, file))

In [58]:
CP = np.reshape(np.array(data['CP']), newshape=(-1, 128, 128))
vec = np.reshape(np.array(data['vecteur_36']) , newshape=(-1, 36))

NN = CP.shape[0]
As = np.zeros(shape=(NN, 32), dtype=complex)
Bs = np.zeros(shape=(NN, 96*96), dtype=float)

amp = fibernet.dataset._utils.make_bellshaped_plane_2d(vec[0].reshape((6,6)))
amp = amp.flatten()

for i in range(NN):
    phi = vec[i]
    phi = amp * np.exp(1j * phi)
    phi = np.delete(vec[i], [0, 5, 30, 35])
    img = CP[i][16:112, 16:112].flatten().astype('float')
    As[i,:] = phi
    Bs[i,:] = np.sqrt(img/(np.power(2, 16) - 1))

In [76]:
%load_ext autoreload
%autoreload 2

N = 5000
idx = np.random.permutation(np.arange(NN))
As_crop = As[idx[:N], :]
Bs_crop = Bs[idx[:N], :]

X_est = pim.pim_tmr(As_crop, Bs_crop, max_iter=10000, tol=20e-9, tol_stag=1e-2, max_stag=10)
B_est = np.abs(np.dot(As_crop, X_est))
print(f"MSE Bs: {pim.mse(Bs_crop, B_est):0.3e}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
i=    0  mse_glob=4.344e-09  mse_convs=4.250e-09  mse_acts=2.163e-08  act=   50  convs= 9166
i=    1  mse_glob=4.338e-09  mse_convs=4.278e-09  mse_acts=2.158e-08  act=   32  convs= 9184
i=    2  mse_glob=4.335e-09  mse_convs=4.291e-09  mse_acts=2.146e-08  act=   24  convs= 9192
i=    3  mse_glob=4.334e-09  mse_convs=4.299e-09  mse_acts=2.129e-08  act=   19  convs= 9197
i=    4  mse_glob=4.333e-09  mse_convs=4.311e-09  mse_acts=2.141e-08  act=   12  convs= 9204
i=    5  mse_glob=4.332e-09  mse_convs=4.317e-09  mse_acts=2.158e-08  act=    8  convs= 9208
i=    6  mse_glob=4.332e-09  mse_convs=4.319e-09  mse_acts=2.139e-08  act=    7  convs= 9209
i=    7  mse_glob=4.331e-09  mse_convs=4.324e-09  mse_acts=2.196e-08  act=    4  convs= 9212
i=    8  mse_glob=4.331e-09  mse_convs=4.325e-09  mse_acts=2.226e-08  act=    3  convs= 9213
i=    9  mse_glob=4.331e-09  mse_convs=4.325e-09  mse_acts=2.194e-08  act= 

In [15]:
import matplotlib.pyplot as plt

i = 120
A_init = As_crop[i,:]
Y_model = np.dot(X_est.T, A_init).reshape((96,96))
B_model = np.abs(Y_model)
B_init = Bs_crop[i,:].reshape((96,96))

fig, axs = plt.subplots(1,4, figsize=(17,3))
pl0 = axs[0].imshow(np.square(B_init), cmap="hot")
pl1 = axs[1].imshow(np.square(B_model), cmap="hot")
pl2 = axs[2].imshow(np.square(B_model) - np.square(B_init), cmap="RdBu")
pl3 = axs[3].imshow(np.angle(Y_model), cmap="twilight")
plt.colorbar(pl0, ax=axs[0])
plt.colorbar(pl1, ax=axs[1])
plt.colorbar(pl2, ax=axs[2])
plt.colorbar(pl3, ax=axs[3])
axs[0].set_title(r"$B_{init}^2$")
axs[1].set_title(r"$B_{model}^2$")
axs[2].set_title(r"Error: $B_{model}^2 - B_{init}^2$")
axs[3].set_title(r"$\angle Y_{model}$")
[ axs[i].axis('off') for i in range(4) ]

NameError: name 'As_crop' is not defined